In [28]:
import pandas as pd
from collections import Counter
import tensorflow as tf
from tqdm import tqdm
from tffm import TFFMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.preprocessing import LabelEncoder
import scipy.sparse as sp

In [29]:
lb=LabelEncoder()

In [30]:
def LabelEncoding_Cat(df):
    df=df.copy()
    mappingTable = {}
    Cat_Var=[i for i in df.select_dtypes('object').columns.tolist() if i != 'Label']
    for col in Cat_Var:
        mappingTable[col]={}
        rowDist = df[col].unique()
        df[col]=lb.fit_transform(df[col].astype('str'))
        idx = df[col].unique()
        for i, j in zip(idx, rowDist):
            mappingTable[col][j] = i 
        print('Transfrom ', col)
    return df ,mappingTable

In [31]:
def FillNa(df):
    df=df.copy()
    Num_Features=df.select_dtypes(['float','int']).columns.tolist()
    Str_Features=df.select_dtypes(['object']).columns.tolist()
    df[Str_Features]=df[Str_Features].fillna('NAN')
    for i in Num_Features:
        df[i]=df[i].fillna(0)
    return df

In [32]:
df = pd.read_csv('train.txt',header=None,delimiter='\t',nrows=100000)
column = ['Label'] + ['I'+str(i+1) for i in range(13)] + ['C'+str(i+1) for i in range(26)]
df.columns = column

In [33]:
df.head()

,Label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NaN,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655
2,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,...,8efede7f,3412118d,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
3,0,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,0.0,...,1e88c74f,74ef3502,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN
4,0,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,0.0,...,1e88c74f,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN


In [34]:
df = FillNa(df)

In [35]:
df, Mapping= LabelEncoding_Cat(df)

Transfrom  C1
Transfrom  C2
Transfrom  C3
Transfrom  C4
Transfrom  C5
Transfrom  C6
Transfrom  C7
Transfrom  C8
Transfrom  C9
Transfrom  C10
Transfrom  C11
Transfrom  C12
Transfrom  C13
Transfrom  C14
Transfrom  C15
Transfrom  C16
Transfrom  C17
Transfrom  C18
Transfrom  C19
Transfrom  C20
Transfrom  C21
Transfrom  C22
Transfrom  C23
Transfrom  C24
Transfrom  C25
Transfrom  C26


In [36]:
Counter(df.Label)

Counter({0: 77337, 1: 22663})

In [37]:
X_all = df.loc[:,'I1':].astype('float32').values
y_all = df.loc[:,'Label'].astype('float32').values

In [38]:
X_tr, X_te, y_tr, y_te = train_test_split(X_all, y_all, random_state=7, test_size=0.3)

In [39]:
X_tr.shape

(70000, 39)

In [40]:
# only CSR format supported
X_tr_sparse = sp.csr_matrix(X_tr)
X_te_sparse = sp.csr_matrix(X_te)

In [41]:
order = 3
model = TFFMRegressor(
    order=order, 
    rank=3, 
    optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), 
    n_epochs=50, 
    batch_size=1024,
    init_std=0.001,
    reg=0.01,
    input_type='sparse',
    seed=42
)

In [ ]:
model.fit(X_tr_sparse, y_tr, show_progress=True)
predictions = model.predict(X_te_sparse)
print('[order={}] accuracy: {}'.format(order, accuracy_score(y_te, predictions)))
model.destroy()

 76%|███████▌  | 38/50 [03:15<01:01,  5.13s/epoch]